In [23]:
import requests
import pandas as pd
import json
import psycopg2
from sqlalchemy import create_engine


In [ ]:
def save_table(df,name):
    import io
    df.head(0).to_sql(f'{name}', engine, if_exists='replace',index=False) #truncates the table

    conn = engine.raw_connection()
    cur = conn.cursor()
    output = io.StringIO()
    df.to_csv(output, sep='\t', header=False, index=False)
    output.seek(0)
    contents = output.getvalue()
    cur.copy_from(output, f'{name}' , null="") # null values become ''
    conn.commit()
    return

In [3]:
engine = create_engine('postgresql+psycopg2://username:secret@db:5432/database')
conn = psycopg2.connect(database="database",user="username", password="secret",host="db", port="5432")
cur = conn.cursor()

In [5]:
data = 'https://api.covidtracking.com/v1/states/daily.json'
d =requests.get(data)
df = pd.read_json(d.text)


In [ ]:
save_table('daily_covid',df)

In [11]:
#https://github.com/nytimes/covid-19-data
data = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
df = pd.read_csv(data)
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [12]:
save_table('base_covid',df)

In [ ]:
#docs https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/cc-est2019-alldata.pdf
data = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/asrh/cc-est2019-alldata.csv'
df = pd.read_csv(data,encoding='ISO-8859-1')
df.head()

In [ ]:
save_table('population',df)

In [16]:
data = 'https://overflow.solutions/wp-content/uploads/2020/05/Poverty_County_18.csv'
df = pd.read_csv(data)
df.head()

,Geo_Name,Total_Pop,Child_Pop,Elder_Pop,Pov_Rate,Pov_Rate_Child,Pov_Rate_Elder,State_FIPS,County_FIPS
0,"Washington County, Mississippi",47086,12349,6953,33.3,48.1,17.1,28,151
1,"Perry County, Mississippi",12028,2809,2072,18.7,25.6,16.6,28,111
2,"Choctaw County, Mississippi",8321,1844,1740,23.3,38.3,13.1,28,19
3,"Itawamba County, Mississippi",23480,5134,4005,15.4,18.3,13.0,28,57
4,"Carroll County, Mississippi",10129,1969,2249,14.8,18.1,14.9,28,15


In [21]:
save_table(df,'poverty')

In [27]:
#https://overflow.solutions/demographic-traits/education/what-u-s-counties-are-the-most-educated/
data = 'https://overflow.solutions/wp-content/uploads/2020/04/Educ_Attain_County_18.csv'
df = pd.read_csv(data)
df.head()

,Geo_Name,25Plus_Pop,HS_plus,Bach_plus,Adanced_Degree,State_FIPS,County_FIPS
0,"Washington County, Mississippi",30308.0,78.5,18.8,7.3,28,151
1,"Perry County, Mississippi",8198.0,81.3,11.0,2.5,28,111
2,"Choctaw County, Mississippi",5799.0,80.4,17.7,6.9,28,19
3,"Itawamba County, Mississippi",15684.0,78.5,13.4,4.9,28,57
4,"Carroll County, Mississippi",7248.0,81.7,14.5,5.0,28,15


In [28]:
save_table(df,'education')

In [34]:
import requests
#website_url = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')
table = soup.find('table',{'class':'wikitable sortable'})
df = pd.read_html(str(table))
df = pd.DataFrame(df[0])
df.head()

,Rank,County or county-equivalent,"State, federal district or territory",Per capitaincome,Medianhouseholdincome,Medianfamilyincome,Population,Number ofhouseholds
0,1,New York County,New York,"$62,498","$69,659","$84,627",1605272.0,736192.0
1,2,Arlington,Virginia,"$62,018","$103,208","$139,244",214861.0,94454.0
2,3,Falls Church City,Virginia,"$59,088","$120,000","$152,857",12731.0,5020.0
3,4,Marin,California,"$56,791","$90,839","$117,357",254643.0,102912.0
4,5,Alexandria City,Virginia,"$54,608","$85,706","$107,511",143684.0,65369.0


In [35]:
save_table(df,'income')